## 1. 간단한 분류모델 구현하기
- 포유류와 조류 구분하기 : 털과 날개가 있냐를 기준으로 분류
- 털과 날개가 있으면 1 , 없으면 0

In [3]:
import numpy as np
import tensorflow as tf

In [4]:
# 털과 날개에 대한 data
# [털,날개] -> [기타,포유류,조류]
x_data=np.array([[0,0],[1,0],[1,1],[0,0],[0,0],[0,1]])
y_data=np.array([[1,0,0],[0,1,0],[0,0,1],[1,0,0],[1,0,0],[0,0,1]])

In [6]:
X=tf.placeholder(tf.float32)
Y=tf.placeholder(tf.float32)

In [7]:
W=tf.Variable(tf.random_uniform([2,3],-1.,-1.))
b=tf.Variable(tf.zeros([3]))

In [8]:
L=tf.add(tf.matmul(X,W),b)
L=tf.nn.relu(L)

In [9]:
model=tf.nn.softmax(L) # 배열내의 결과값을 전체합이 1이 되도록 만들어줌

In [10]:
# 손실함수 지정 (교차엔트로피 계산식)
cost=tf.reduce_mean(-tf.reduce_sum(Y*tf.log(model),axis=1))

In [20]:
# 학습
## optimizer는 경사하강법
optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op=optimizer.minimize(cost)

# session 초기화
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

for step in range(100):
    sess.run(train_op,feed_dict={X: x_data, Y:y_data})
    if (step+1)%10 ==0 :
        print(step+1,sess.run(cost,feed_dict={X:x_data,Y:y_data}))

10 1.0986123
20 1.0986123
30 1.0986123
40 1.0986123
50 1.0986123
60 1.0986123
70 1.0986123
80 1.0986123
90 1.0986123
100 1.0986123


In [21]:
# 학습결과 확인
prediction=tf.argmax(model,axis=1) # argmax : 가장큰값의 인덱스(레이블)을 출력해줌
target=tf.argmax(Y,axis=1)
print('예측값',sess.run(prediction,feed_dict={X:x_data}))
print('실제값',sess.run(target,feed_dict={Y:y_data}))

예측값 [0 0 0 0 0 0]
실제값 [0 1 2 0 0 2]


In [22]:
# 정확도 출력
is_correct=tf.equal(prediction,target)
accuracy=tf.reduce_mean(tf.cast(is_correct,tf.float32))
print('정확도: %.2f'%sess.run(accuracy*100, feed_dict={X:x_data,Y:y_data}) )

정확도: 50.00


## 2. 심층 신경망 구현하기

In [23]:
W1=tf.Variable(tf.random_uniform([2,10],-1,1)) # [2,10] : 특징, 은닉층 뉴런수
W2=tf.Variable(tf.random_uniform([10,3],-1,1)) # [10,3] : 은닉층 뉴런수, 분류수
b1=tf.Variable(tf.zeros([10]))
b2=tf.Variable(tf.zeros([3]))

In [24]:
L1=tf.add(tf.matmul(X,W1),b1) # 노드 만들기
L1=tf.nn.relu(L1)

In [25]:
model=tf.add(tf.matmul(L1,W2),b2)

In [26]:
# 손실함수 , optimizer 
# 텐서플로가 기본으로 제공하는 교차 엔트로피 함수를 이용.
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,logits=model))
optimizer=tf.train.AdamOptimizer(learning_rate=0.01)
train_op=optimizer.minimize(cost)